# Deep Q-Learning 

Install dependencies for AI gym to run properly (shouldn't take more than a minute). If running on google cloud or running locally, only need to run once. Colab may require installing everytime the vm shuts down.

In [ ]:
!pip3 install gym pyvirtualdisplay
!sudo apt-get install -y xvfb python-opengl ffmpeg

In [ ]:
!pip3 install --upgrade setuptools --user
!pip3 install ez_setup 
!pip3 install gym[atari] 
!pip3 install gym[accept-rom-license] 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import find_max_lives, check_live, get_frame, get_init_state
from model import DQN
from config import *

import matplotlib.pyplot as plt

## Understanding the environment

In the following cell, we initialize our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://www.gymlibrary.dev/environments/atari/breakout/. 

In breakout, we will use 3 actions "fire", "left", and "right". "fire" is only used to reset the game when a life is lost, "left" moves the agent left and "right" moves the agent right.

In [3]:
env = gym.make('BreakoutDeterministic-v4')
state = env.reset()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [4]:
number_lives = find_max_lives(env)
state_size = env.observation_space.shape
action_size = 3 #fire, left, and right

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. Once you've created a working DQN agent, use the code in agent.py to create a double DQN agent in __agent_double.py__. Set the flag "double_dqn" to True to train the double DQN agent.

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [5]:
double_dqn = False # set to True if using double DQN agent

if double_dqn:
    from agent_double import Agent
else:
    from agent import Agent

agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0

### Main Training Loop

In this training loop, we do not render the screen because it slows down training signficantly. To watch the agent play the game, run the code in next section "Visualize Agent Performance"

In [ ]:
rewards, episodes = [], []
best_eval_reward = 0
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    state = env.reset()
    next_state = state
    life = number_lives

    get_init_state(history, state, HISTORY_SIZE)

    while not done:
        step += 1
        frame += 1

        # Perform a fire action if ball is no longer on screen to continue onto next life
        if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:
            action = 0
        else:
            action = agent.get_action(np.float32(history[:4, :, :]) / 255.)
        state = next_state
        next_state, reward, done, _, info = env.step(action + 1)
        
        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['lives'])

        life = info['lives']
        r = reward

        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
        # Start training after random sample generation
        if(frame >= train_frame):
            agent.train_policy_net(frame)
            # Update the target network only for Double DQN only
            if (frame % update_target_network_frequency)== 0:
                agent.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]
            
        if done:
            evaluation_reward.append(score)
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.xlabel('Episodes')
            pylab.ylabel('Rewards') 
            pylab.title('Episodes vs Reward')
            pylab.savefig("./save_graph/breakout_dqn.png") # save graph for training visualization
            
            # every episode, plot the play time
            print("epis:", e, "  score:", score, "  mem len:",
                  len(agent.memory), "  epsilon:", round(agent.epsilon, 4), "   steps:", step,
                  "   lr:", round(agent.optimizer.param_groups[0]['lr'], 7), "    reward:", round(np.mean(evaluation_reward), 2))

            # if the mean of scores of last 100 episode is bigger than 5 save model
            ### Change this save condition to whatever you prefer ###
            if np.mean(evaluation_reward) > 5 and np.mean(evaluation_reward) > best_eval_reward:
                torch.save(agent.policy_net, "./save_model/breakout_dqn.pth")
                best_eval_reward = np.mean(evaluation_reward)


/tmp/ipykernel_309641/1376515593.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:
/tmp/ipykernel_309641/1376515593.py:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:


epis: 0   score: 2.0   mem len: 218   epsilon: 1.0    steps: 218    lr: 0.0001     reward: 2.0
epis: 1   score: 1.0   mem len: 369   epsilon: 1.0    steps: 151    lr: 0.0001     reward: 1.5
epis: 2   score: 2.0   mem len: 567   epsilon: 1.0    steps: 198    lr: 0.0001     reward: 1.67
epis: 3   score: 1.0   mem len: 739   epsilon: 1.0    steps: 172    lr: 0.0001     reward: 1.5
epis: 4   score: 1.0   mem len: 889   epsilon: 1.0    steps: 150    lr: 0.0001     reward: 1.4
epis: 5   score: 1.0   mem len: 1039   epsilon: 1.0    steps: 150    lr: 0.0001     reward: 1.33
epis: 6   score: 0.0   mem len: 1162   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.14
epis: 7   score: 0.0   mem len: 1284   epsilon: 1.0    steps: 122    lr: 0.0001     reward: 1.0
epis: 8   score: 0.0   mem len: 1407   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 0.89
epis: 9   score: 2.0   mem len: 1625   epsilon: 1.0    steps: 218    lr: 0.0001     reward: 1.0
epis: 10   score: 2.0   mem len: 1822   e

epis: 84   score: 0.0   mem len: 15676   epsilon: 1.0    steps: 122    lr: 0.0001     reward: 1.54
epis: 85   score: 1.0   mem len: 15845   epsilon: 1.0    steps: 169    lr: 0.0001     reward: 1.53
epis: 86   score: 0.0   mem len: 15968   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.52
epis: 87   score: 2.0   mem len: 16189   epsilon: 1.0    steps: 221    lr: 0.0001     reward: 1.52
epis: 88   score: 0.0   mem len: 16312   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.51
epis: 89   score: 1.0   mem len: 16480   epsilon: 1.0    steps: 168    lr: 0.0001     reward: 1.5
epis: 90   score: 0.0   mem len: 16602   epsilon: 1.0    steps: 122    lr: 0.0001     reward: 1.48
epis: 91   score: 0.0   mem len: 16725   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.47
epis: 92   score: 3.0   mem len: 16996   epsilon: 1.0    steps: 271    lr: 0.0001     reward: 1.48
epis: 93   score: 1.0   mem len: 17147   epsilon: 1.0    steps: 151    lr: 0.0001     reward: 1.48
epis: 94   

epis: 167   score: 3.0   mem len: 30814   epsilon: 1.0    steps: 230    lr: 0.0001     reward: 1.44
epis: 168   score: 1.0   mem len: 30982   epsilon: 1.0    steps: 168    lr: 0.0001     reward: 1.43
epis: 169   score: 1.0   mem len: 31134   epsilon: 1.0    steps: 152    lr: 0.0001     reward: 1.4
epis: 170   score: 0.0   mem len: 31257   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.39
epis: 171   score: 2.0   mem len: 31454   epsilon: 1.0    steps: 197    lr: 0.0001     reward: 1.4
epis: 172   score: 3.0   mem len: 31700   epsilon: 1.0    steps: 246    lr: 0.0001     reward: 1.4
epis: 173   score: 2.0   mem len: 31897   epsilon: 1.0    steps: 197    lr: 0.0001     reward: 1.42
epis: 174   score: 0.0   mem len: 32020   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.41
epis: 175   score: 0.0   mem len: 32143   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.4
epis: 176   score: 1.0   mem len: 32294   epsilon: 1.0    steps: 151    lr: 0.0001     reward: 1.39
epis

epis: 250   score: 1.0   mem len: 46102   epsilon: 1.0    steps: 150    lr: 0.0001     reward: 1.53
epis: 251   score: 0.0   mem len: 46225   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.53
epis: 252   score: 0.0   mem len: 46348   epsilon: 1.0    steps: 123    lr: 0.0001     reward: 1.53
epis: 253   score: 1.0   mem len: 46498   epsilon: 1.0    steps: 150    lr: 0.0001     reward: 1.54
epis: 254   score: 2.0   mem len: 46678   epsilon: 1.0    steps: 180    lr: 0.0001     reward: 1.54
epis: 255   score: 0.0   mem len: 46800   epsilon: 1.0    steps: 122    lr: 0.0001     reward: 1.51
epis: 256   score: 3.0   mem len: 47029   epsilon: 1.0    steps: 229    lr: 0.0001     reward: 1.49
epis: 257   score: 1.0   mem len: 47197   epsilon: 1.0    steps: 168    lr: 0.0001     reward: 1.5
epis: 258   score: 1.0   mem len: 47369   epsilon: 1.0    steps: 172    lr: 0.0001     reward: 1.51
epis: 259   score: 0.0   mem len: 47491   epsilon: 1.0    steps: 122    lr: 0.0001     reward: 1.49
e

...

epis: 949   score: 5.0   mem len: 174655   epsilon: 0.8522    steps: 306    lr: 0.0001     reward: 2.07
epis: 950   score: 3.0   mem len: 174881   epsilon: 0.8517    steps: 226    lr: 0.0001     reward: 2.08
epis: 951   score: 6.0   mem len: 175292   epsilon: 0.8509    steps: 411    lr: 0.0001     reward: 2.14
epis: 952   score: 3.0   mem len: 175539   epsilon: 0.8504    steps: 247    lr: 0.0001     reward: 2.16
epis: 953   score: 2.0   mem len: 175737   epsilon: 0.85    steps: 198    lr: 0.0001     reward: 2.16
epis: 954   score: 0.0   mem len: 175860   epsilon: 0.8498    steps: 123    lr: 0.0001     reward: 2.12
epis: 955   score: 3.0   mem len: 176086   epsilon: 0.8493    steps: 226    lr: 0.0001     reward: 2.11
epis: 956   score: 1.0   mem len: 176255   epsilon: 0.849    steps: 169    lr: 0.0001     reward: 2.08
epis: 957   score: 0.0   mem len: 176378   epsilon: 0.8488    steps: 123    lr: 0.0001     reward: 2.07
epis: 958   score: 2.0   mem len: 176577   epsilon: 0.8484    steps

epis: 1028   score: 5.0   mem len: 191693   epsilon: 0.8184    steps: 341    lr: 0.0001     reward: 2.26
epis: 1029   score: 2.0   mem len: 191891   epsilon: 0.8181    steps: 198    lr: 0.0001     reward: 2.24
epis: 1030   score: 1.0   mem len: 192059   epsilon: 0.8177    steps: 168    lr: 0.0001     reward: 2.23
epis: 1031   score: 6.0   mem len: 192380   epsilon: 0.8171    steps: 321    lr: 0.0001     reward: 2.28
epis: 1032   score: 2.0   mem len: 192579   epsilon: 0.8167    steps: 199    lr: 0.0001     reward: 2.29
epis: 1033   score: 2.0   mem len: 192777   epsilon: 0.8163    steps: 198    lr: 0.0001     reward: 2.31
epis: 1034   score: 0.0   mem len: 192900   epsilon: 0.8161    steps: 123    lr: 0.0001     reward: 2.29
epis: 1035   score: 3.0   mem len: 193146   epsilon: 0.8156    steps: 246    lr: 0.0001     reward: 2.3
epis: 1036   score: 0.0   mem len: 193269   epsilon: 0.8153    steps: 123    lr: 0.0001     reward: 2.3
epis: 1037   score: 1.0   mem len: 193420   epsilon: 0.81

epis: 1107   score: 6.0   mem len: 209831   epsilon: 0.7825    steps: 351    lr: 4e-05     reward: 2.6
epis: 1108   score: 3.0   mem len: 210062   epsilon: 0.7821    steps: 231    lr: 4e-05     reward: 2.59
epis: 1109   score: 2.0   mem len: 210243   epsilon: 0.7817    steps: 181    lr: 4e-05     reward: 2.59
epis: 1110   score: 4.0   mem len: 210521   epsilon: 0.7812    steps: 278    lr: 4e-05     reward: 2.63
epis: 1111   score: 4.0   mem len: 210813   epsilon: 0.7806    steps: 292    lr: 4e-05     reward: 2.65
epis: 1112   score: 3.0   mem len: 211038   epsilon: 0.7801    steps: 225    lr: 4e-05     reward: 2.65
epis: 1113   score: 8.0   mem len: 211506   epsilon: 0.7792    steps: 468    lr: 4e-05     reward: 2.73
epis: 1114   score: 2.0   mem len: 211723   epsilon: 0.7788    steps: 217    lr: 4e-05     reward: 2.7
epis: 1115   score: 7.0   mem len: 212115   epsilon: 0.778    steps: 392    lr: 4e-05     reward: 2.77
epis: 1116   score: 1.0   mem len: 212285   epsilon: 0.7777    step

epis: 1186   score: 2.0   mem len: 230906   epsilon: 0.7408    steps: 199    lr: 4e-05     reward: 3.57
epis: 1187   score: 2.0   mem len: 231088   epsilon: 0.7404    steps: 182    lr: 4e-05     reward: 3.55
epis: 1188   score: 8.0   mem len: 231520   epsilon: 0.7396    steps: 432    lr: 4e-05     reward: 3.61
epis: 1189   score: 4.0   mem len: 231812   epsilon: 0.739    steps: 292    lr: 4e-05     reward: 3.62
epis: 1190   score: 1.0   mem len: 231981   epsilon: 0.7387    steps: 169    lr: 4e-05     reward: 3.59
epis: 1191   score: 2.0   mem len: 232163   epsilon: 0.7383    steps: 182    lr: 4e-05     reward: 3.59
epis: 1192   score: 5.0   mem len: 232485   epsilon: 0.7377    steps: 322    lr: 4e-05     reward: 3.6
epis: 1193   score: 2.0   mem len: 232683   epsilon: 0.7373    steps: 198    lr: 4e-05     reward: 3.57
epis: 1194   score: 5.0   mem len: 233009   epsilon: 0.7366    steps: 326    lr: 4e-05     reward: 3.59
epis: 1195   score: 4.0   mem len: 233266   epsilon: 0.7361    ste

epis: 1265   score: 2.0   mem len: 252894   epsilon: 0.6973    steps: 197    lr: 4e-05     reward: 4.16
epis: 1266   score: 4.0   mem len: 253137   epsilon: 0.6968    steps: 243    lr: 4e-05     reward: 4.12
epis: 1267   score: 6.0   mem len: 253494   epsilon: 0.6961    steps: 357    lr: 4e-05     reward: 4.12
epis: 1268   score: 3.0   mem len: 253723   epsilon: 0.6956    steps: 229    lr: 4e-05     reward: 4.14
epis: 1269   score: 3.0   mem len: 253971   epsilon: 0.6951    steps: 248    lr: 4e-05     reward: 4.07
epis: 1270   score: 4.0   mem len: 254246   epsilon: 0.6946    steps: 275    lr: 4e-05     reward: 4.05
epis: 1271   score: 4.0   mem len: 254539   epsilon: 0.694    steps: 293    lr: 4e-05     reward: 4.05
epis: 1272   score: 3.0   mem len: 254788   epsilon: 0.6935    steps: 249    lr: 4e-05     reward: 4.0
epis: 1273   score: 2.0   mem len: 254986   epsilon: 0.6931    steps: 198    lr: 4e-05     reward: 3.98
epis: 1274   score: 3.0   mem len: 255230   epsilon: 0.6926    ste

# Visualize Agent Performance

BE AWARE THIS CODE BELOW MAY CRASH THE KERNEL IF YOU RUN THE SAME CELL TWICE.

Please save your model before running this portion of the code.

In [ ]:
torch.save(agent.policy_net, "./save_model/breakout_dqn_latest.pth")

In [ ]:
from gym.wrappers import RecordVideo # If importing monitor raises issues, try using `from gym.wrappers import RecordVideo`
import glob
import io
import base64

from IPython.display import HTML
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display

# Displaying the game live
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % ("Agent Playing",step, info))
    plt.axis('off')

    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf())
    
# Recording the game and replaying the game afterwards
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env):
    env = RecordVideo(env, './video')
    return env

In [ ]:
display = Display(visible=0, size=(300, 200))
display.start()

# Load agent
# agent.load_policy_net("./save_model/breakout_dqn.pth")
agent.epsilon = 0.0 # Set agent to only exploit the best action

env = gym.make('BreakoutDeterministic-v4')
env = wrap_env(env)

done = False
score = 0
step = 0
state = env.reset()
next_state = state
life = number_lives
history = np.zeros([5, 84, 84], dtype=np.uint8)
get_init_state(history, state)

while not done:
    
    # Render breakout
    env.render()
#     show_state(env,step) # uncommenting this provides another way to visualize the game

    step += 1
    frame += 1

    # Perform a fire action if ball is no longer on screen
    if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:
        action = 0
    else:
        action = agent.get_action(np.float32(history[:4, :, :]) / 255.)
    state = next_state
    
    next_state, reward, done, _, info = env.step(action + 1)
        
    frame_next_state = get_frame(next_state)
    history[4, :, :] = frame_next_state
    terminal_state = check_live(life, info['ale.lives'])
        
    life = info['ale.lives']
    r = np.clip(reward, -1, 1) 
    r = reward

    # Store the transition in memory 
    agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
    # Start training after random sample generation
    score += reward
    
    history[:4, :, :] = history[1:, :, :]
env.close()
show_video()
display.stop()